In [7]:
%load_ext autoreload
%autoreload 2
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from agents.knn import *
from settings import settings

def reduce_actions(actions):
    s = np.unique(np.array([a['steer'] for a in actions]))
    s.sort()
    steer_values = np.unique(np.concatenate((
        np.linspace(-1, -0.1, 15),
        np.linspace(-0.1, 0 , 6),
        np.linspace(0, 0.1, 6),
        np.linspace(0.1, 1, 15),
    )))
    return [
        {
            **a,
            'steer': steer_values[
                np.argmin(
                    np.abs(
                        steer_values - a['steer']
                    )
                 )
            ]
        } 
        for a in actions
    ]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
data = parse_dataset('/home/i279079/important_logs')
state_keys = settings['state_keys']
states, actions = extract_parameters(data)
Y, actions = unify_actions(actions)
X = np.array(states)
Y = np.array(Y)

In [24]:
state_keys

['speedX', 'speedY', 'speedZ', 'angle', 'trackPos', 'track', 'wheelSpinVel']

In [29]:
state_keys = settings['state_keys'] 
states, actions = extract_parameters(data, action_keys=[
    'accel',
    'brake',
    'steer']
)
# self.actions = actions
# Reducinf actions set
# reduced_actons = reduce_actions(actions)
Y, actions = unify_actions(reduced_actons)
Y = to_categorical(Y)

X = np.array(states)
X, Y = sklearn.utils.shuffle(X, np.array(Y))


In [30]:
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.20)

In [32]:

scaler = StandardScaler()  
scaler.fit(X_train)  
# classifier = MLPClassifier(hidden_layer_sizes=(3000,2000, 500), max_iter=10000)
# classifier = sklearn.ensemble.ExtraTreesClassifier(300, min_samples_leaf=20)
model = Sequential()
model.add(Dense(5000, input_dim=28, activation='relu'))
model.add(Dense(2000, activation='relu'))
model.add(Dense(500, activation='sigmoid'))
model.add(Dense(
#     1,
    len(actions),
    activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

classifier = model

print(scaler.transform(X_train).shape)
classifier.fit(
    scaler.transform(X_train),
    Y_train,
    batch_size=1000,
    epochs=150,
    validation_data=(X_test, Y_test)
) 

(24333, 28)
Train on 24333 samples, validate on 6084 samples
Epoch 1/150
24333/24333 [==============================] - 0s - loss: 2.8782 - acc: 0.5922 - val_loss: 2.2360 - val_acc: 0.6164
Epoch 2/150
24333/24333 [==============================] - 0s - loss: 1.7728 - acc: 0.6167 - val_loss: 2.4457 - val_acc: 0.6164
Epoch 3/150
24333/24333 [==============================] - 0s - loss: 1.6081 - acc: 0.6167 - val_loss: 2.5931 - val_acc: 0.6164
Epoch 4/150
24333/24333 [==============================] - 0s - loss: 1.5162 - acc: 0.6167 - val_loss: 2.7068 - val_acc: 0.6164
Epoch 5/150
24333/24333 [==============================] - 0s - loss: 1.4416 - acc: 0.6168 - val_loss: 2.6834 - val_acc: 0.6164
Epoch 6/150
24333/24333 [==============================] - 0s - loss: 1.3767 - acc: 0.6329 - val_loss: 2.6644 - val_acc: 0.6164
Epoch 7/150
24333/24333 [==============================] - 0s - loss: 1.3067 - acc: 0.6594 - val_loss: 2.6616 - val_acc: 0.6183
Epoch 8/150
24333/24333 [==================

24333/24333 [==============================] - 0s - loss: 0.4311 - acc: 0.8581 - val_loss: 4.4545 - val_acc: 0.6180
Epoch 65/150
24333/24333 [==============================] - 0s - loss: 0.4247 - acc: 0.8650 - val_loss: 4.4133 - val_acc: 0.6182
Epoch 66/150
24333/24333 [==============================] - 0s - loss: 0.4110 - acc: 0.8689 - val_loss: 4.3888 - val_acc: 0.6180
Epoch 67/150
24333/24333 [==============================] - 0s - loss: 0.3987 - acc: 0.8714 - val_loss: 4.3751 - val_acc: 0.6182
Epoch 68/150
24333/24333 [==============================] - 0s - loss: 0.4098 - acc: 0.8660 - val_loss: 4.3817 - val_acc: 0.6179
Epoch 69/150
24333/24333 [==============================] - 0s - loss: 0.3940 - acc: 0.8744 - val_loss: 4.4925 - val_acc: 0.6182
Epoch 70/150
24333/24333 [==============================] - 0s - loss: 0.3743 - acc: 0.8809 - val_loss: 4.3875 - val_acc: 0.6182
Epoch 71/150
24333/24333 [==============================] - 0s - loss: 0.3796 - acc: 0.8767 - val_loss: 4.3678

24333/24333 [==============================] - 0s - loss: 0.2526 - acc: 0.9139 - val_loss: 5.0460 - val_acc: 0.6170
Epoch 127/150
24333/24333 [==============================] - 0s - loss: 0.2464 - acc: 0.9178 - val_loss: 5.0820 - val_acc: 0.6183
Epoch 128/150
24333/24333 [==============================] - 0s - loss: 0.2503 - acc: 0.9146 - val_loss: 4.9590 - val_acc: 0.6177
Epoch 129/150
24333/24333 [==============================] - 0s - loss: 0.2428 - acc: 0.9177 - val_loss: 5.1678 - val_acc: 0.6174
Epoch 130/150
24333/24333 [==============================] - 0s - loss: 0.2444 - acc: 0.9169 - val_loss: 5.0604 - val_acc: 0.6175
Epoch 131/150
24333/24333 [==============================] - 0s - loss: 0.2473 - acc: 0.9153 - val_loss: 5.0484 - val_acc: 0.6170
Epoch 132/150
24333/24333 [==============================] - 0s - loss: 0.2444 - acc: 0.9180 - val_loss: 5.0856 - val_acc: 0.6183
Epoch 133/150
24333/24333 [==============================] - 0s - loss: 0.2600 - acc: 0.9117 - val_loss:

In [33]:
classifier.evaluate(scaler.transform(X_train), Y_train) , classifier.evaluate(scaler.transform(X_test), Y_test) 

5472/6084 [=========================>....] - ETA: 0s

([0.21657571621245003, 0.93116344059507661],
 [0.57473213134981316, 0.81656804733727806])

In [ ]:
classifier.metrics_names

In [17]:
classifier.predict_classes(
    np.array(X_test[3:4])
)

1/1 [==============================] - 0s


array([59])

In [ ]:
np.array(X_test[3:4]),np.array(Y_test[3:4])

In [5]:

# (reduce_actions(actions))

In [34]:
def dump():
    path = 'saved_model'
    print(f'Dumping model to: {path}')
    np.savez(
        'saved_model/parameters.npz',
        **{
            'actions': actions,
            'state_keys': state_keys,
#             'reduce_actions': reduce_actions
        }
    
    )
    classifier.save('saved_model/classifier')
    joblib.dump(scaler, 'saved_model/scaler')
    
dump()
    

Dumping model to: saved_model
